In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import *
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [24]:
type = "sort"
data = pd.read_csv(f"./data/{type}_results.csv")


avg = data.groupby(['array_type', 'element_size']).mean().reset_index()
SQ_data = avg[avg["array_type"] == "SQ"]
sliced_data = avg[avg["array_type"] == "sliced"]
mono_data = avg[avg["array_type"] == "mono"]
base_data = avg[avg["array_type"] == "base"]
HAT_data = avg[avg["array_type"] == "HAT"]
MS2 = avg[avg["array_type"] == "MSarray_2"]
MS3 = avg[avg["array_type"] == "MSarray_3"]
MS4 = avg[avg["array_type"] == "MSarray_4"]

In [4]:
SQ_data

,array_type,element_size,time
512,SQ,16,0.098292
513,SQ,32,0.097430
514,SQ,48,0.098965
515,SQ,64,0.097403
516,SQ,80,0.097437
...,...,...,...
635,SQ,1984,0.192050
636,SQ,2000,0.194836
637,SQ,2016,0.186031
638,SQ,2032,0.191536


In [10]:
def visualise_position(r1, r2, r3, r4, r5, r6, r7):
    p = figure(title = 'run time of different array types', plot_width = 900, x_axis_label = 'size of each element', y_axis_label = 'avergae time')
    p.line(r1.element_size, r1.time, color = 'blue', legend_label = "SQarray")
    p.line(r2.element_size, r2.time, color = 'red', legend_label = "sliced")
    p.line(r3.element_size, r3.time, color = 'green', legend_label = "monolithic")
    p.line(r4.element_size, r4.time, color = 'green', legend_label = "HAT")
    p.legend.location = "top_left"
    show(p)
visualise_position(SQ_data, sliced_data, mono_data)

TypeError: visualise_position() missing 4 required positional arguments: 'r4', 'r5', 'r6', and 'r7'

In [25]:
def visualise_with_base(r1, r2, r3, r4, r5, r6, r7, r8):
    p = figure(title = 'run time of different array types', plot_width = 900, x_axis_label = 'size of each element', y_axis_label = 'avergae time')
    p.line(r1.element_size, r1.time, color = 'blue', legend_label = "SQarray")
    p.line(r2.element_size, r2.time, color = 'red', legend_label = "sliced")
    p.line(r3.element_size, r3.time, color = 'green', legend_label = "monolithic")
    p.line(r4.element_size, r4.time, color = 'black', legend_label = "base")
    p.line(r5.element_size, r5.time, color = 'yellow', legend_label = "HAT")
    p.line(r6.element_size, r6.time, color = 'orange', legend_label = "MS, r=2")
    p.line(r7.element_size, r7.time, color = 'purple', legend_label = "MS, r=3")
    p.line(r8.element_size, r8.time, color = 'indigo', legend_label = "MS, r=4")
    p.legend.location = "top_left"
    show(p)
visualise_with_base(SQ_data, sliced_data, mono_data, base_data, HAT_data, MS2, MS3, MS4)

In [17]:
volatility = data.groupby(['array_type', 'element_size']).std().reset_index()
SQ_vol = volatility[volatility["array_type"] == "SQ"]
sliced_vol = volatility[volatility["array_type"] == "sliced"]
mono_vol = volatility[volatility["array_type"] == "mono"]
base_vol = volatility[volatility["array_type"] == "base"]

visualise_with_base(SQ_vol, sliced_vol, mono_vol, base_vol)

TypeError: visualise_with_base() missing 4 required positional arguments: 'r5', 'r6', 'r7', and 'r8'

In [5]:
def visualise_log(r1, r2, r3, r4):
    p = figure(title = 'run time of different array types', plot_width = 900, x_axis_label = 'size of each element', y_axis_label = 'avergae time')
    p.line(np.log2(r1.element_size), np.log2(r1.time), color = 'blue', legend_label = "SQarray")
    p.line(np.log2(r2.element_size), np.log2(r2.time), color = 'red', legend_label = "sliced")
    p.line(np.log2(r3.element_size), np.log2(r3.time), color = 'green', legend_label = "monolithic")
    p.line(np.log2(r4.element_size), np.log2(r4.time), color = 'black', legend_label = "base")
    p.legend.location = "top_left"
    show(p)
visualise_log(SQ_data, sliced_data, mono_data, base_data)